In [1]:
from docling_sdg.qa.generate import Generator
from docling_sdg.qa.base import GenerateOptions

In [18]:
generate_options = GenerateOptions(api_key="fake", project_id="project_id")
generate_options.api_key = "fake"
generate_options.model_id = "mixtral"
gen = Generator(generate_options=generate_options)

In [20]:
from docling_sdg.qa.base import initialize_llm

llm = initialize_llm(generate_options)

from docling_sdg.qa.utils import ChatAgent

agent = ChatAgent(llm=llm)

from docling_sdg.qa.base import QaChunk
chunk = QaChunk(meta={"doc_items": [{"self_ref":"#/CfL","label": "text"}], "chunk_id": "","doc_id": ""}, text="George Washington (February 22, 1732 [O.S. February 11, 1731][a] – December 14, 1799) was a Founding Father and the first president of the United States, serving from 1789 to 1797. As commander of the Continental Army, Washington led Patriot forces to victory in the American Revolutionary War against the British Empire. He is commonly known as the Father of His Country for his role in bringing about American independence.")

gen.generate_from_chunks([chunk])

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 14463.12it/s]

status=<Status.SUCCESS: 'success'> time_taken=0.0036449432373046875 output=PosixPath('docling_sdg_generated_qac.jsonl') num_qac=0


NameError: name 'docling_sdg_generated_qac' is not defined

In [21]:
with open("docling_sdg_generated_qac.jsonl", "rt") as f:
    print(f.readlines())

['{"context": "George Washington (February 22, 1732 [O.S. February 11, 1731][a] – December 14, 1799) was a Founding Father and the first president of the United States, serving from 1789 to 1797. As commander of the Continental Army, Washington led Patriot forces to victory in the American Revolutionary War against the British Empire. He is commonly known as the Father of His Country for his role in bringing about American independence.", "question": "What was George Washington\'s role before becoming the first president of the United States?", "answer": "George Washington was a Founding Father and the commander of the Continental Army, leading Patriot forces to victory in the American Revolutionary War against the British Empire.", "short_answer": null, "retrieved_context": false, "generated_question": true, "generated_answer": true, "created": "2025-03-24T18:09:33.706032+00:00", "user": null, "model": "mixtral", "paths": [""], "advanced": null, "labels": {"scope": null, "alignment": 